### IMPORTS

In [181]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from gspread_pandas import Spread, conf

### CREDENCIALES

In [182]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### QUERIES

In [189]:
q = '''SELECT uc.country AS Country,
       uc.city AS City,
       uc.based AS Based,
       COUNT(DISTINCT uc.user) AS Users
FROM `peya-argentina.user_santiago_curat.city_usuarios` AS uc
GROUP BY 1,2,3
ORDER BY 1,2,3,4 DESC'''

In [197]:
# Descargo la data
bq_crudo = pd.io.gbq.read_gbq(q, project_id='peya-argentina', dialect='standard')


Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2797/2797 [00:00<00:00, 5667.49rows/s]


In [198]:
# Copio la base
crudo = bq_crudo.copy()

### TRABAJO

In [199]:
# Doy formato a las columnas
crudo['Users'] = crudo['Users'].astype(int)

In [200]:
# Creo una PT para ver el Based
based = crudo.pivot_table(index=['Based'],values=['Users'],aggfunc='sum',fill_value=0).reset_index()
# Coloco los porcentajes
based['Users MM'] = based['Users'] / 1000000
based['Total MM'] = based['Users'].sum() / 1000000
based['%'] = round(based['Users'] / based['Users'].sum() * 100,2)

In [178]:
# Creo una PT por pais
paises = crudo.pivot_table(index=['Country','City'],values=['Users'],aggfunc='sum',fill_value=0).reset_index()
# Filtro Argentina
arg = paises[paises['Country'] == 'Argentina'].copy()
arg['Users MM'] = arg['Users'] / 1000000
arg['Total MM'] = arg['Users'].sum() / 1000000
arg['%'] = round(arg['Users'] / arg['Users'].sum() * 100,2)
arg.sort_values(by=['Users'],ascending=False,inplace=True)
arg['Acum'] = arg['%'].cumsum(axis=0)
# Filtro Chile
chi = paises[paises['Country'] == 'Chile'].copy()
chi['Users MM'] = chi['Users'] / 1000000
chi['Total MM'] = chi['Users'].sum() / 1000000
chi['%'] = round(chi['Users'] / chi['Users'].sum() * 100,2)
chi.sort_values(by=['Users'],ascending=False,inplace=True)
chi['Acum'] = chi['%'].cumsum(axis=0)
# Filtro Uruguay
uru = paises[paises['Country'] == 'Uruguay'].copy()
uru['Users MM'] = uru['Users'] / 1000000
uru['Total MM'] = uru['Users'].sum() / 1000000
uru['%'] = round(uru['Users'] / uru['Users'].sum() * 100,2)
uru.sort_values(by=['Users'],ascending=False,inplace=True)
uru['Acum'] = uru['%'].cumsum(axis=0)

In [201]:
based

,Based,Users,Users MM,Total MM,%
0,-,36061527,36.061527,62.71161,57.50
1,ADDRESS BASED,21192724,21.192724,62.71161,33.79
2,MOST ADDRESSED BASED,634633,0.634633,62.71161,1.01
3,ORDER BASED,4822726,4.822726,62.71161,7.69
